# Pandas 支持的时间格式：
+ 时间格式字符串
+ np.datetime64
+ datetime.datetime

In [2]:
import datetime

import numpy as np
import pandas as pd

In [3]:
dt = pd.to_datetime(['1/1/2018', np.datetime64('2018-01-01'), datetime.datetime(2018, 1, 1)])
dt

DatetimeIndex(['2018-01-01', '2018-01-01', '2018-01-01'], dtype='datetime64[ns]', freq=None)

In [5]:
dt = pd.date_range(start='2018-01-01', periods=3, freq='H')
dt

DatetimeIndex(['2018-01-01 00:00:00', '2018-01-01 01:00:00',
               '2018-01-01 02:00:00'],
              dtype='datetime64[ns]', freq='H')

In [9]:
# 设置时区

dt = dt.tz_localize('UTC')
dt

DatetimeIndex(['2018-01-01 00:00:00+00:00', '2018-01-01 01:00:00+00:00',
               '2018-01-01 02:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='H')

In [10]:
# 切换时区

dt.tz_convert('US/Pacific')

DatetimeIndex(['2017-12-31 16:00:00-08:00', '2017-12-31 17:00:00-08:00',
               '2017-12-31 18:00:00-08:00'],
              dtype='datetime64[ns, US/Pacific]', freq='H')

# 按指定频率重采样

In [11]:
_idx = pd.date_range('2018-01-01', periods=5, freq='H')
ts = pd.Series(range(len(_idx)), index=_idx)
ts

2018-01-01 00:00:00    0
2018-01-01 01:00:00    1
2018-01-01 02:00:00    2
2018-01-01 03:00:00    3
2018-01-01 04:00:00    4
Freq: H, dtype: int64

In [12]:
ts.resample('2H').mean()

2018-01-01 00:00:00    0.5
2018-01-01 02:00:00    2.5
2018-01-01 04:00:00    4.0
Freq: 2H, dtype: float64

# 用绝对或相对时间差计算日期与时间

In [13]:
friday = pd.Timestamp('2018-01-05')
friday.day_name()

'Friday'

In [14]:
saturday = friday + pd.Timedelta('1 day')
saturday.day_name()

'Saturday'

In [15]:
monday = friday + pd.offsets.BDay()
monday.day_name()

'Monday'

# Pandas 支持的4种常见时间概念
+ 日期时间(Datetime)：带时区的日期时间，类似于标准库的 datetime.datetime
+ 时间差(Timedelta): 绝对时间周期，类似于标准库的 datetime.timedelta
+ 时间段(Timespan)：在某一时点以指定频率定义的时间跨度
+ 日期偏移(Dateoffset)：与日历运算对应的时间段，类似 dateutil 的 dateutil.relativedelta.relativedelta

时间概念 | 标量类 | 数组类 | Pandas数据类型 | 主要构建方法
:--- | :--- | :--- | :--- | :---
Datetimes | Timestamp | DatetimeIndex | datetime64[ns] 或 datetime64[ns,tz] | to_datetime 或 date_range
Timedeltas | Timedelta | TimedeltaIndex | timedelta64[ns] | to_timedelta 或 timedelta_range
Timespans | Period | PeriodIndex | period[freq] | Period 或 period_range
Dateoffsets | DateOffset | None | None | DateOffset

In [17]:
pd.Series(pd.date_range('1/1/2011', freq='M', periods=3))

0   2011-01-31
1   2011-02-28
2   2011-03-31
dtype: datetime64[ns]

In [21]:
pd.Series(pd.timedelta_range(start='1 day', freq='6H', periods=3))

0   1 days 00:00:00
1   1 days 06:00:00
2   1 days 12:00:00
dtype: timedelta64[ns]

In [16]:
pd.Series(pd.period_range('1/1/2011', freq='M', periods=3))

0    2011-01
1    2011-02
2    2011-03
dtype: period[M]

# NaT
> Pandas用`NaT`表示日期时间、时间差及时间段的空值，代表了缺失日期或空日期的值，类似于浮点数的`np.nan`

In [22]:
pd.Timestamp(pd.NaT)

NaT

In [23]:
pd.Timedelta(pd.NaT)

NaT

In [24]:
pd.Period(pd.NaT)

NaT

In [25]:
# 与 np.nan 一样，pd.NaT 不等于 pd.NaT 

pd.NaT == pd.NaT

False

# Timestamp 与 Period 可以用作索引
> 作为索引的 Timestamp 与 Period 列表则被强制转换为对应的 DatetimeIndex 与 PeriodIndex

In [26]:
periods = [pd.Period('2012-01'), pd.Period('2012-02'), pd.Period('2012-03')]
ts = pd.Series(np.random.randn(3), periods)
ts.index

PeriodIndex(['2012-01', '2012-02', '2012-03'], dtype='period[M]', freq='M')

# 转换时间戳
> `to_datetime` 函数用于转换字符串、纪元式及混合的日期`Series`或日期列表。转换`Series`时，返回的是具有相同的索引的`Series`，日期时间列表则会被转换为`DatetimeIndex`

In [27]:
# 转换Series

pd.to_datetime(pd.Series(['Jul 31, 2009', '2010-01-10', None]))

0   2009-07-31
1   2010-01-10
2          NaT
dtype: datetime64[ns]

In [29]:
# 转换日期时间列表

pd.to_datetime(['2005/11/23', '2010.12.31'])

DatetimeIndex(['2005-11-23', '2010-12-31'], dtype='datetime64[ns]', freq=None)

In [30]:
pd.to_datetime(['04-01-2012 10:00'], dayfirst=True)

DatetimeIndex(['2012-01-04 10:00:00'], dtype='datetime64[ns]', freq=None)

In [31]:
pd.to_datetime(['04-01-2012 10:00'], dayfirst=False)

DatetimeIndex(['2012-04-01 10:00:00'], dtype='datetime64[ns]', freq=None)

In [33]:
# 如果不能把第一个数解析为日，就会以 dayfirst 为 False 进行解析

pd.to_datetime(['14-01-2012', '01-14-2012'], dayfirst=True)

DatetimeIndex(['2012-01-14', '2012-01-14'], dtype='datetime64[ns]', freq=None)

## `to_datetime` 转换单个字符串时，返回的是单个 Timestamp。Timestamp 仅支持字符串输入，不支持 dayfirst、format 等字符串解析选项

In [34]:
pd.to_datetime('2010/11/12')

Timestamp('2010-11-12 00:00:00')

## 创建`DatetimeIndex`时，传递字符串`infer`即可推断索引的频率 (前提：频率是有规律的)

In [36]:
pd.DatetimeIndex(['2018-01-01', '2018-01-03', '2018-01-05'], freq='infer')

DatetimeIndex(['2018-01-01', '2018-01-03', '2018-01-05'], dtype='datetime64[ns]', freq='2D')

## 要实现精准转换，除了传递`datetime`字符串，还要指定`format`参数

In [39]:
pd.to_datetime('12-11-2010 00:00')

Timestamp('2010-12-11 00:00:00')

In [40]:
pd.to_datetime('12-11-2010 00:00', format='%d-%m-%Y %H:%M')

Timestamp('2010-11-12 00:00:00')

## Pandas 还可以把 DataFrame 里的整数或字符串列组合成 Timestamp Series

pd.to_datetime 查找列名里日期时间组件的标准名称，包括：
+ 必选：year、month、day
+ 可选：hour、minute、second、millisecond、microsecond、nanosecond

In [45]:
_df = pd.DataFrame({'year': [2015, 2016],
                   'month': [2, 3],
                   'day': [4, 5],
                   'hour': [2, 3]})
pd.to_datetime(_df)

0   2015-02-04 02:00:00
1   2016-03-05 03:00:00
dtype: datetime64[ns]

In [43]:
pd.to_datetime(_df[['year', 'month', 'day']])

0   2015-02-04
1   2016-03-05
dtype: datetime64[ns]

## 无效数据

In [46]:
# 不可解析时，默认值 errors='raise' 会触发错误

pd.to_datetime(['2009/07/31', 'asd'], errors='raise')

ParserError: Unknown string format: asd

In [47]:
# errors='ignore' 返回原始输入

pd.to_datetime(['2009/07/31', 'asd'], errors='ignore')

Index(['2009/07/31', 'asd'], dtype='object')

In [48]:
# errors='coerce' 把无法解析的数据转换为 NaT，即不是时间（Not a Time）

pd.to_datetime(['2009/07/31', 'asd'], errors='coerce')

DatetimeIndex(['2009-07-31', 'NaT'], dtype='datetime64[ns]', freq=None)

## pandas 支持把整数或浮点数纪元时间转换为 Timestamp 与 DatetimeIndex
> 鉴于 Timestamp 对象内部存储方式，这种转换的默认单位是纳秒。不过，一般都会用指定其它时间单位 unit 来存储纪元数据，纪元时间从 origin 参数指定的时点开始计算。

In [49]:
pd.to_datetime([1349720105, 1349806505, 1349892905, 1349979305, 1350065705], unit='s')

DatetimeIndex(['2012-10-08 18:15:05', '2012-10-09 18:15:05',
               '2012-10-10 18:15:05', '2012-10-11 18:15:05',
               '2012-10-12 18:15:05'],
              dtype='datetime64[ns]', freq=None)

In [50]:
pd.to_datetime([1349720105100, 1349720105200, 1349720105300, 
                1349720105400, 1349720105500], unit='ms')

DatetimeIndex(['2012-10-08 18:15:05.100000', '2012-10-08 18:15:05.200000',
               '2012-10-08 18:15:05.300000', '2012-10-08 18:15:05.400000',
               '2012-10-08 18:15:05.500000'],
              dtype='datetime64[ns]', freq=None)

## 把时间戳转换为纪元
> 把 Timestamp 转换为 unix 纪元

In [51]:
stamps = pd.date_range('2012-10-08 18:15:05', periods=4, freq='D')
stamps

DatetimeIndex(['2012-10-08 18:15:05', '2012-10-09 18:15:05',
               '2012-10-10 18:15:05', '2012-10-11 18:15:05'],
              dtype='datetime64[ns]', freq='D')

In [52]:
# 首先与纪元开始时点（1970年1月1日午夜，UTC）相减，然后以 1 秒为时间单位（unit='1s'）取底整除

(stamps - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

Int64Index([1349720105, 1349806505, 1349892905, 1349979305], dtype='int64')

## 时间戳的界限
> Pandas 时间戳的最低单位为纳秒，64 位整数显示的时间跨度约为 584 年，这就是 Timestamp 的界限

In [53]:
pd.Timestamp.min

Timestamp('1677-09-21 00:12:43.145225')

In [54]:
pd.Timestamp.max

Timestamp('2262-04-11 23:47:16.854775807')